In [7]:
import os

os.chdir('/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/michael_dev/RA2_alpine_lads/ra_joint_predictions')

In [8]:
from utils.config import Config

config = Config()

In [9]:
from dataset.joint_val_dataset import hands_wrists_val_dataset

wrists_ds = hands_wrists_val_dataset(config)

wrist_j_data = wrists_ds.create_wrists_joints_dataset_with_validation(config.train_location + '/training.csv', erosion_flag = False)
wrist_e_data = wrists_ds.create_wrists_joints_dataset_with_validation(config.train_location + '/training.csv', erosion_flag = True)

2020-05-12 17:40:10,102;WARNING - Missing permissions to create directory for caching!
2020-05-12 17:40:10,105;WARNING - Missing permissions to create directory for caching!
[{'augment': <function random_brightness_and_contrast at 0x7f3c0d142a60>}, {'augment': <function random_crop at 0x7f3c0d142b70>, 'params': {'min_scale': 0.9}}, {'augment': <function random_gaussian_noise at 0x7f3c0d142bf8>, 'p': 0.2}, {'augment': <function random_rotation at 0x7f3c0d142ae8>, 'params': {'max_rot': 10}}]
2020-05-12 17:40:11,307;WARNING - Missing permissions to create directory for caching!
2020-05-12 17:40:11,310;WARNING - Missing permissions to create directory for caching!
[{'augment': <function random_brightness_and_contrast at 0x7f3c0d142a60>}, {'augment': <function random_crop at 0x7f3c0d142b70>, 'params': {'min_scale': 0.9}}, {'augment': <function random_gaussian_noise at 0x7f3c0d142bf8>, 'p': 0.2}, {'augment': <function random_rotation at 0x7f3c0d142ae8>, 'params': {'max_rot': 10}}]


In [16]:
import numpy as np

from tensorflow import keras
from model.utils.metrics import mae_metric, rmse_metric, class_filter_rmse_metric, softmax_mae_metric, softmax_rmse_metric, class_filter_softmax_rmse_metric

def eval_model(label, dataset, model_paths, max_output, steps, filter_model_paths = [], filter_cutoff = 0.5):
    truths = {
        0: [],
        1: [],
        2: [],
        3: [],
        4: [],
        5: []
    }
    
    preds = {
        0: [],
        1: [],
        2: [],
        3: [],
        4: [],
        5: []
    }
    
    models = [keras.models.load_model(model_path, compile = False) for model_path in model_paths]
    filter_models = [keras.models.load_model(filter_model_path, compile = False) for filter_model_path in filter_model_paths]
    
    for x, y in dataset.take(steps):
        y_preds = np.zeros((x.shape[0], 6))
        
        
        for model in models:
            y_pred = model.predict(x)
            
            for n in range(6):
                y_preds[:, n] += y_pred[n][:, 0]
                
        y_preds = y_preds / len(models)
        
        pred_filter = np.ones((x.shape[0], 6))
        if len(filter_models) > 0:
            filter_preds = np.zeros((x.shape[0], 6))
            
            for filter_model in filter_models:
                f_pred = filter_model.predict(x)
                
                for n in range(6):
                    filter_preds[:, n] += f_pred[n][:, 0]
                
            filter_preds = filter_preds / len(filter_models)
            
            pred_filter[filter_preds < filter_cutoff] = 0
            
        y_preds = np.multiply(y_preds, pred_filter)
       
        for n in range(6):
            truths[n].extend(y[n][:, 0].numpy())
            preds[n].extend(y_preds[:, n])
    
    loss = np.zeros(6)
    rmse = np.zeros(6)
    mae = np.zeros(6)
    filter_rmse = np.zeros(6)
            
    for idx in range(6):
        t_vals = np.array(truths[idx])
        p_vals = np.array(preds[idx])
        
        non0_idx = np.where(t_vals != 0.0)[0]
        
        loss[idx] = np.mean(np.square(t_vals - p_vals))
        rmse[idx] = np.sqrt(np.mean(np.square(t_vals - p_vals)))
        filter_rmse[idx] = np.sqrt(np.mean(np.square(t_vals[non0_idx] - p_vals[non0_idx])))
        mae[idx] = np.mean(np.absolute(t_vals - p_vals))
    
    print('Model:', label)
    print('Loss:', loss)
    print('MAE:', mae)
    print('RMSE:', rmse)
    print('Filter RMSE:', filter_rmse)
    

In [29]:
eval_model('Wrists Narrowing', wrist_j_data[1], ['../trained_models/narrowing/v7/wrists_narrowing_joint_damage_model_complex_rewritten_64bs_60steps_75epochs_adamW_1e3_1e6_changed_size_augs_left_group_test.h5'], 4, 3)

Model: Wrists Narrowing
Loss: [0.32278784 0.55198494 0.32462334 0.57798025 0.38627925 0.34097186]
MAE: [0.28046825 0.41887947 0.26107301 0.42232104 0.27336472 0.23893593]
RMSE: [0.56814421 0.74295689 0.56975726 0.76025012 0.62151367 0.58392796]
Filter RMSE: [0.99737993 1.25377347 1.17989585 1.13696659 1.51776572 1.49668864]


In [30]:
eval_model('Wrists Narrowing (L+R)', wrist_j_data[1], ['../trained_models/narrowing/v7/wrists_narrowing_joint_damage_model_complex_rewritten_64bs_60steps_75epochs_adamW_1e3_1e6_changed_size_augs_left_group_test.h5', '../trained_models/narrowing/v7/wrists_narrowing_joint_damage_model_complex_rewritten_64bs_60steps_75epochs_adamW_1e3_1e6_changed_size_augs_right_group_test.h5'], 4, 3)

Model: Wrists Narrowing (L+R)
Loss: [0.30685485 0.5053486  0.29064872 0.54039065 0.38953567 0.34703918]
MAE: [0.28508981 0.3871709  0.26244075 0.42079286 0.27286552 0.23365479]
RMSE: [0.55394481 0.71087875 0.53911846 0.73511268 0.62412793 0.58910031]
Filter RMSE: [0.97974739 1.20424425 1.08184077 1.08316729 1.54658915 1.52835052]


In [18]:
j_filter_model_paths = ['../trained_models/narrowing/v7/wrists_narrowing_joint_damage_type_model_complex_rewritten_gap_64bs_normsteps_75epochs_adamW_1e3_1e6_changed_size_changed_augs_right_group_test.h5',
'../trained_models/narrowing/v7/wrists_narrowing_joint_damage_type_model_complex_rewritten_gap_64bs_normsteps_75epochs_adamW_1e3_1e6_changed_size_changed_augs_left_group_test.h5',
'../trained_models/narrowing/v7/wrists_narrowing_joint_damage_type_model_complex_rewritten_gap_64bs_normsteps_75epochs_adamW_1e3_1e6_changed_size_changed_augs_test.h5']

eval_model('Wrists Narrowing (L+R)', wrist_j_data[1], ['../trained_models/narrowing/v7/wrists_narrowing_joint_damage_model_complex_rewritten_64bs_60steps_75epochs_adamW_1e3_1e6_changed_size_augs_left_group_test.h5', '../trained_models/narrowing/v7/wrists_narrowing_joint_damage_model_complex_rewritten_64bs_60steps_75epochs_adamW_1e3_1e6_changed_size_augs_right_group_test.h5'], 4, 3, filter_model_paths = j_filter_model_paths, filter_cutoff = 0.3)

Model: Wrists Narrowing (L+R)
Loss: [0.33483676 0.53088192 0.27594038 0.53426211 0.39147263 0.35149788]
MAE: [0.24338233 0.34987574 0.21290689 0.37047457 0.23314459 0.20622156]
RMSE: [0.57865081 0.72861644 0.52530028 0.73093235 0.62567773 0.59287257]
Filter RMSE: [1.0964227  1.25700774 1.17201105 1.16467433 1.58918671 1.5715311 ]


In [27]:
eval_model('Wrists Erosion', wrist_e_data[1], ['../trained_models/erosion/v7/wrists_erosion_joint_damage_model_complex_rewritten_64bs_60steps_75epochs_adamW_1e3_1e6_changed_size_augs_left_group_test.h5'], 5, 3)

Model: Wrists Erosion
Loss: [0.71637571 0.82234632 0.96798352 0.74732168 0.84979826 0.90601827]
MAE: [0.33714562 0.44104488 0.37611096 0.5351008  0.33147945 0.40366783]
RMSE: [0.84638981 0.90683313 0.98386154 0.86447769 0.92184503 0.95184992]
Filter RMSE: [2.22640934 1.93773458 2.64631255 1.4178807  2.53894543 1.86734771]


In [24]:
e_filter_model_paths = ['../trained_models/erosion/v7/wrists_erosion_joint_damage_type_model_complex_rewritten_gap_64bs_normsteps_75epochs_adamW_1e3_1e6_changed_size_changed_augs_left_group_test.h5',
'../trained_models/erosion/v7/wrists_erosion_joint_damage_type_model_complex_rewritten_gap_64bs_normsteps_75epochs_adamW_1e3_1e6_changed_size_changed_augs_right_group_test.h5',
'../trained_models/erosion/v7/wrists_erosion_joint_damage_type_model_complex_rewritten_gap_64bs_normsteps_75epochs_adamW_1e3_1e6_changed_size_changed_augs_test.h5']

e_model_paths = ['../trained_models/erosion/v7/wrists_erosion_joint_damage_model_complex_rewritten_64bs_60steps_75epochs_adamW_1e3_1e6_changed_size_augs_test.h5',
'../trained_models/erosion/v7/wrists_erosion_joint_damage_model_complex_rewritten_64bs_60steps_75epochs_adamW_1e3_1e6_changed_size_augs_right_group_test.h5',
'../trained_models/erosion/v7/wrists_erosion_joint_damage_model_complex_rewritten_64bs_60steps_75epochs_adamW_1e3_1e6_changed_size_augs_left_group_test.h5']

In [25]:
eval_model('Wrists Erosion (L+R)', wrist_e_data[1], e_model_paths, 5, 3)

Model: Wrists Erosion (L+R)
Loss: [0.77320023 0.71897733 0.94289979 0.71439601 0.78759132 0.79518461]
MAE: [0.33861831 0.40775129 0.37140257 0.5449272  0.31097745 0.39617305]
RMSE: [0.87931805 0.84792531 0.97103027 0.8452195  0.88746342 0.89173125]
Filter RMSE: [2.33193844 1.81942968 2.57001109 1.4022256  2.4452377  1.79598721]


In [26]:
eval_model('Wrists Erosion (L+R)', wrist_e_data[1], e_model_paths, 5, 3, filter_model_paths = e_filter_model_paths, filter_cutoff = 0.2)

Model: Wrists Erosion (L+R)
Loss: [0.827713   0.72783649 0.93703968 0.69540145 0.85885555 0.81903887]
MAE: [0.29558598 0.36886477 0.33992006 0.47794153 0.29616748 0.37260831]
RMSE: [0.90978734 0.85313334 0.9680081  0.83390734 0.9267446  0.90500766]
Filter RMSE: [2.44083862 1.84320986 2.57201611 1.41514349 2.5624469  1.836018  ]
